Importing all require modules and packages

In [1]:
# Importing all modelues and packages required
import sqlite3 as sql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import requests
import re
import time
import os

# Preprocessing
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,StackingClassifier
from lightgbm import LGBMClassifier

# Tuning
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support as score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

In [5]:
import opendatasets as od

In [8]:
od.download("https://www.kaggle.com/competitions/titanic/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading titanic.zip to .\titanic


100%|██████████| 34.1k/34.1k [00:00<00:00, 990kB/s]


Extracting archive .\titanic/titanic.zip to .\titanic


In [ ]:
# retrieve the db file from the web
response = requests.get('https://techassessment.blob.core.windows.net/aiap12-assessment-data/attrition.db')

# Check whether request is successful or not: 200 = successful
print(response.status_code)